In [89]:
import numpy as np
import matplotlib.pyplot as plt
import sjoert.stellar
import pandas as pd
import os
from tqdm import tqdm
import forced_phot as fp #import flux_jy, chi2, gaussian, chi2_peak_finder
import json
from numpyencoder import NumpyEncoder #by Hunter M. Allen (https://pypi.org/project/numpyencoder/)
from astropy.io import ascii

param_path = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\params_good.txt'
param_df = pd.read_csv(param_path, sep=",")

refmag_path = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\refmags_good.txt'
refmag_df = pd.read_csv(refmag_path, sep=",")

param_df.columns

Index(['ZTF_ID', 'F_p', 'F_p_err', 't_0', 't_0_err', 'sigma_rise',
       'sigma_rise_err', 'tau_dec', 'tau_dec_err', 'T', 'T_err', 'F_0g',
       'F_0g_err', 'F_0r', 'F_0r_err', 'log10_F_p', 'log10_F_p_err',
       'log10_sigma_rise', 'log10_sigma_rise_err', 'log10_tau_dec',
       'log10_tau_dec_err', 'log10_T', 'log10_T_err', 'nu_0', 't0_cc', 'Fp_cc',
       'chi2_dof'],
      dtype='object')

Filtering bad fits

In [90]:
sigma_rise = param_df['sigma_rise'].to_numpy()
tau_dec = param_df['tau_dec'].to_numpy()

sigma_rise_err = param_df['sigma_rise_err'].to_numpy()

#rules for filtering out bad fits
bad_log10sigma_error_mask = param_df['log10_sigma_rise_err'] < 0.2 
bad_log10tau_error_mask = param_df['log10_tau_dec_err'] <  0.2

param_df = param_df[bad_log10sigma_error_mask * bad_log10tau_error_mask]
refmag_df = refmag_df[bad_log10sigma_error_mask * bad_log10tau_error_mask]

flux2mag = lambda F: sjoert.stellar.flux2mag(F*1e-6) #bc F in uJy
def flux2mag_err(F,sigma_F):
    F = F*1e-6
    sigma_F = sigma_F * 1e-6
    return 2.5/(np.log(10) * (F)) * sigma_F

Fp,Fp_err = param_df['F_p'].to_numpy(),param_df['F_p_err'].to_numpy()

SNR_mask = param_df['log10_F_p_err'] < 0.3
Fp,Fp_err = Fp[SNR_mask],Fp_err[SNR_mask]

# print(pd.DataFrame({'mag_p':flux2mag(Fp),'mag_p_err':flux2mag_err(Fp,Fp_err),'mag_ref_g':refmag_df['refmag_g'].values[SNR_mask],'mag_ref_r':refmag_df['refmag_r'].values[SNR_mask]}).to_markdown())
magp = flux2mag(Fp)
diff_g = magp - refmag_df['refmag_g'].values[SNR_mask]
diff_r = magp - refmag_df['refmag_r'].values[SNR_mask]

param_df = param_df[SNR_mask]
refmag_df = refmag_df[SNR_mask]

In [91]:
ztfs = param_df['ZTF_ID'].to_numpy()
ztfs.shape

(9604,)

In [92]:
class_file_path = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\class_info.dat'

class_info = ascii.read(class_file_path)
# class_ztfs = np.array(class_info['ztf_id'])
# ztf_mask_small = np.isin(ztfs,class_ztfs)
# ztf_mask_large = np.isin(class_ztfs,ztfs)
# ztf_mask_small.sum()

In [93]:
class_info

ztf_id,RA,Dec,redshift,MBH,MBH_ref,classification
str12,float64,float64,float64,float64,str20,str17
ZTF17aaaacjo,88.12306625,14.44325405,0.0,0.0,--,long dummy string
ZTF17aaaazwi,73.2609679,11.9668683,0.0,0.0,--,--
ZTF17aaadkwx,32.5596005,-1.8135499,0.0,0.0,--,--
ZTF17aaaduqw,7.0192117,42.5484892,0.0,0.0,--,--
ZTF17aaaecja,311.10945065,51.1478584,0.0,0.0,--,--
ZTF17aaaephv,359.15167335,70.8050378,0.0,0.0,--,--
ZTF17aaaeqnq,19.84207925,32.0440752,0.0,0.0,--,--
ZTF17aaagpwv,54.04777105,46.18035165,0.0,0.0,--,--
ZTF17aaaheis,94.8376514,31.3210664,0.0,0.0,--,--


In [94]:
mask = class_info['classification'].data.mask
classifications,class_indx = class_info['classification'][np.invert(mask)][1:], np.where(np.invert(mask)[1:]) #drop the first since it's just a dummy string
classifications = np.array(classifications)

#new array where instances of 
classifications_binary = classifications.copy()
classifications_binary[classifications == 'AGN'] = 1
classifications_binary[classifications != 'AGN'] = 0
classifications_binary = classifications_binary.astype(np.int32)




In [95]:
filled_classifications = class_info['classification'].copy()[1:].filled('Unknown')
print(np.unique(filled_classifications,return_counts=True))

(<Column name='classification' dtype='str17' length=4>
    AGN
     SN
    TDE
Unknown, array([ 1944,   173,    58, 12082], dtype=int64))


In [96]:
class_info_filled = class_info.copy()
class_info_filled['classification'] = class_info_filled['classification'].filled('Unknown')
class_info_filled['classification'][0] = 'Unknown'
class_info_filled

ztf_id,RA,Dec,redshift,MBH,MBH_ref,classification
str12,float64,float64,float64,float64,str20,str17
ZTF17aaaacjo,88.12306625,14.44325405,0.0,0.0,--,Unknown
ZTF17aaaazwi,73.2609679,11.9668683,0.0,0.0,--,Unknown
ZTF17aaadkwx,32.5596005,-1.8135499,0.0,0.0,--,Unknown
ZTF17aaaduqw,7.0192117,42.5484892,0.0,0.0,--,Unknown
ZTF17aaaecja,311.10945065,51.1478584,0.0,0.0,--,Unknown
ZTF17aaaephv,359.15167335,70.8050378,0.0,0.0,--,Unknown
ZTF17aaaeqnq,19.84207925,32.0440752,0.0,0.0,--,Unknown
ZTF17aaagpwv,54.04777105,46.18035165,0.0,0.0,--,Unknown
ZTF17aaaheis,94.8376514,31.3210664,0.0,0.0,--,Unknown


In [102]:
relevant_class_info = class_info_filled[['ztf_id','MBH','MBH_ref','classification']]

param_path = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\params_good.txt'
param_df = pd.read_csv(param_path, sep=",")
refmag_path = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\refmags_good.txt'
refmag_df = pd.read_csv(refmag_path, sep=",")


sigma_rise = param_df['sigma_rise'].to_numpy()
tau_dec = param_df['tau_dec'].to_numpy()

sigma_rise_err = param_df['sigma_rise_err'].to_numpy()

#rules for filtering out bad fits
bad_log10sigma_error_mask = param_df['log10_sigma_rise_err'] < 0.2 
bad_log10tau_error_mask = param_df['log10_tau_dec_err'] <  0.2

param_df = param_df[bad_log10sigma_error_mask * bad_log10tau_error_mask]
refmag_df = refmag_df[bad_log10sigma_error_mask * bad_log10tau_error_mask]
print(param_df.shape)
Fp,Fp_err = param_df['F_p'].to_numpy(),param_df['F_p_err'].to_numpy()
SNR_mask = param_df['log10_F_p_err'] < 0.2
Fp,Fp_err = Fp[SNR_mask],Fp_err[SNR_mask]

# print(pd.DataFrame({'mag_p':flux2mag(Fp),'mag_p_err':flux2mag_err(Fp,Fp_err),'mag_ref_g':refmag_df['refmag_g'].values[SNR_mask],'mag_ref_r':refmag_df['refmag_r'].values[SNR_mask]}).to_markdown())
magp = flux2mag(Fp)
diff_g = magp - refmag_df['refmag_g'].values[SNR_mask]
diff_r = magp - refmag_df['refmag_r'].values[SNR_mask]

param_df = param_df[SNR_mask]

ztfmask = np.isin(np.array(relevant_class_info['ztf_id']),np.array(param_df['ZTF_ID']))
print(sum(ztfmask))

ascii.write(relevant_class_info[ztfmask],os.path.join(os.getcwd()[:62],'Data','relevant_class_info.dat'),overwrite=True)


(9727, 27)
[False False False ...  True  True False]
